<a href="https://colab.research.google.com/github/Mohit-Mithra/AuthNet/blob/master/BottleBody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
import numpy as np

In [ ]:
from numpy import load
from keras import losses

In [ ]:
def oversample(dataset):
    pos_over_pos=dataset[0:5,:,:]
    over_sample=[]
    for i in range(20):
        if i == 0:
            over_sample=pos_over_pos
        else:
            over_sample=np.concatenate((over_sample,pos_over_pos))
        print(over_sample.shape)
        
    neg_over_neg = dataset[5:15,:,:]
    print(neg_over_neg.shape)
    over_sample_neg=[]
    for i in range(10):
        if i == 0:
            over_sample_neg=neg_over_neg
        else:
            over_sample_neg=np.concatenate((over_sample_neg,neg_over_neg))   
    X_data_over_sample_neg=np.concatenate((dataset,over_sample_neg))
    X_data_over_sample=np.concatenate((X_data_over_sample_neg,over_sample))
    
    print(X_data_over_sample.shape)
    return X_data_over_sample

In [ ]:
def labeller():  

    Y_train_p=np.ones((5,1))
    Y_train_n=np.zeros((10,1))
    Y_data=np.concatenate((Y_train_p,Y_train_n))
    Y_data_oversample_neg=np.concatenate((Y_data,np.zeros((100,1))))
    Y_data_oversample_without_storedneg=np.concatenate((Y_data_oversample_neg,np.ones((100,1))))
    Y_data_oversample=np.concatenate((Y_data_oversample_without_storedneg,np.zeros((200,1))))
    return Y_data_oversample

In [ ]:
def trainer(X_train, y_train):

    model = Sequential()

    input_shape=(X_train.shape[1],X_train.shape[2])

    model.add(LSTM(units=11,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(0.2))

    model.add(LSTM(units=16,return_sequences=True))
    model.add(Dropout(0.2))


    model.add(LSTM(units=16,return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=16,input_shape=input_shape)) 
    model.add(Dropout(0.2))

    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy')
    model.fit(X_train,y_train,epochs=50,batch_size=75)
    
    return model